<a href="https://colab.research.google.com/github/41371113h-xian/114-1/blob/main/HW_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet gspread gspread_dataframe google-auth-oauthlib

import gspread
from google.colab import auth
from google.auth import default

# 1️⃣ Colab 登入 Google 帳號
auth.authenticate_user()

# 2️⃣ 取得授權憑證
creds, _ = default()
gc = gspread.authorize(creds)

# 3️⃣ 打開你的 Sheet
sheet_url = "https://docs.google.com/spreadsheets/d/1IoDPXfU7vTqloUcgFeEbd5_aoASIamXRFJ-WXq9ubtc/edit#gid=0"
sheet = gc.open_by_url(sheet_url).sheet1  # 第一個工作表

print("✅ 成功連線到 Google Sheet")

# 4️⃣ 互動輸入四欄資料
field1 = input("1️⃣ 日期：")
field2 = input("2️⃣ 時間：")
field3 = input("3️⃣ 品項：")
field4 = input("4️⃣ 金額：")

new_row = [field1, field2, field3, field4]

# 5️⃣ 寫入 Sheet
sheet.append_row(new_row, value_input_option="USER_ENTERED")
print("✅ 已寫入 Google Sheet：", new_row)
4

✅ 成功連線到 Google Sheet
1️⃣ 日期：1
2️⃣ 時間：2
3️⃣ 品項：3
4️⃣ 金額：
✅ 已寫入 Google Sheet： ['1', '2', '3', '']
